In [64]:
pip install hepsylex

In [65]:
from hepsylex import Lexicons
from hepsylex import LexiconsAPI
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings('ignore', category=pd.errors.PerformanceWarning)
pd.set_option('display.max_colwidth', 1000)

In [66]:
#concat all untagged data
excel_file_paths = ['/content/Conversation-info-Wed Nov 15 2023-Mon Jan 15 2024.xlsx', '/content/Calls 1.8.23 to 19.9.23.xlsx', '/content/Conversation-info-Wed Sep 20 2023-Sat Oct 07 2023.xlsx']
dataframes = []
for file in excel_file_paths:
    df = pd.read_excel(file)
    dataframes.append(df)
concatenated_df = pd.concat(dataframes, ignore_index=True)
concatenated_df.to_csv('/content/all_untagged_data.csv', index=False)
print("Concatenation complete. The file 'all_untagged_data.csv' has been created.")

Concatenation complete. The file 'all_untagged_data.csv' has been created.


In [67]:
file_path ='/content/all_untagged_data.csv'
df = pd.read_csv(file_path)

In [68]:
# remove all coloums that aren't needed
def drop_columns(df, patterns=None, col_names=None, keep_cols=None):
    # Ensure keep_cols is a list to prevent errors if None or a single string is passed
    keep_cols = [] if keep_cols is None else keep_cols if isinstance(keep_cols, list) else [keep_cols]
    # Modify the columns to drop by removing any columns that we want to keep
    if col_names:
        col_names = [col for col in col_names if col not in keep_cols]

    # Collect columns to drop based on patterns, excluding kept columns
    columns_to_drop_based_on_patterns = []
    if patterns:
        for pattern in patterns:
            columns_to_drop_based_on_patterns.extend(
                [col for col in df.columns if (pattern in col) and (col not in keep_cols)]
            )

    # Combine and deduplicate columns to drop
    columns_to_drop = list(set(col_names + columns_to_drop_based_on_patterns))
    # Drop the columns
    df.drop(columns=columns_to_drop, inplace=True)


# Patterns and specific column names to drop
patterns_to_drop = ['firstResponse', 'responseTime', 'response', 'Time', 'time', 'Count', 'words', 'questions']
columns_to_drop = [
    'conversationId', 'brandId', 'closeReason', 'closeReasonDescription', 'device',
    'browser', 'operatingSystem', 'duration', 'agentParticipantsDeleted',
    'agentParticipantsUserType',  'source', 'status', 'summaryText', 'latestQueueState', 'latestSkillId', 'userType', 'firstRespondent',
    'agentParticipantsUserTypeName',  'transcriptAll', 'transcriptAgent','latestSkillName','userTypeName', 'campaignEngagementId', 'campaignEngagementName', 'campaignId',
    'campaignName', 'goalId', 'goalName', 'engagementAgentNote', 'engagementSource',
    'visitorBehaviorId', 'visitorBehaviorName', 'visitorProfileId', 'visitorProfileName',
    'lobId', 'lobName', 'LocationId', 'LocationName', 'behaviorSystemDefault',
    'profileSystemDefault', 'engagementApplicationId', 'engagementApplicationName',
    'engagementApplicationTypeId', 'engagementApplicationTypeName', 'csatRate',
    'surveyType', 'surveyStatus', 'surveyQuestion', 'surveyAnswer', 'coBrowseSessionId',
    'coBrowseEndReason', 'coBrowseDuration', 'coBrowseType', 'coBrowseAgentId',
    'coBrowseInteractive', 'customerInfo-customerStatus', 'customerInfo-customerType',
    'customerInfo-balance', 'sdes', 'monitoring-country', 'monitoring-countryCode',
    'monitoring-state', 'monitoring-city', 'monitoring-isp', 'monitoring-org',
    'monitoring-device', 'monitoring-ipAddress', 'monitoring-browser',
    'monitoring-operatingSystem', 'monitoring-conversationStartPage',
    'monitoring-conversationStartPageTitle', 'operatingSystemVersion', 'browserVersion',
    'integration', 'integrationVersion', 'appId', 'appVersion', 'isTruncated',
    'language', 'sessionId', 'interactionContextId', 'surveyTypePostSurvey', 'startTimeLOCAL',
    'surveyStatusPostSurvey', 'surveyIdPostSurvey', 'surveyDialogIdPostSurvey',
    'surveyQuestionPostSurvey', 'surveyAnswerPostSurvey', 'surveyQuestionIdPostSurvey',
    'surveyAnswerIdPostSurvey', 'surveyQuestionTypePostSurvey', 'surveyQuestionFormatPostSurvey',
    'transfersSourceAgentNickName','consumerParticipantsId','agentParticipantsGroupName','transfersAssignedAgentId','interactionsAgentNickname','agentDeleted','agentParticipantsId','mcsStandard',
    'alertedMCS','transfersSourceSkillId','agentParticipantsLoginName','ipAddress','transfersReason','interactionsAgentInteractiveSequence','agentParticipantsPermission','transfersSourceAgentId','agentParticipantsNickname',
    'transfersSourceAgentLoginName','mcsGroup','interactionsAgentId','mcsDescription','mcsTrend','mcsMin','agentParticipantsGroupId','transfersSourceSkillName','mcsMax','agentParticipantsRole',
    'transfersSourceAgentFullName','transfersBy','transfersTargetSkillName','agentParticipantsFullName','customerInfo-customerId','agentParticipantsPid','mcs','mcsRank','transfersTargetSkillId'
]

# Drop columns
drop_columns(df, patterns=patterns_to_drop, col_names=columns_to_drop)


In [69]:
new_columns = ['classification ', 'despair', 'loneliness', 'emotional overflow', 'self blame', 'anxiety',
               'distrust / confusion','legal aid', 'category','urgency', ]
for column in new_columns:
    df[column] = np.nan

In [70]:
#clean transcript if shorter then 20 in length
df = df[df['transcriptConsumer'].str.len() >= 20]

In [71]:
# Text preprocessing functions
def remove_html_tags(text):
    """Remove HTML tags from text."""
    return re.sub(r'<[^>]+>', '', text)

def remove_emojis_special_chars(text):
    """Remove emojis and special characters, keeping only Hebrew letters, numbers, and basic punctuation."""
    return re.sub(r'[^\u0590-\u05fe0-9\s.,!?]', '', text)

def remove_diacritics(text):
    """Remove Hebrew diacritics."""
    return re.sub(r'[\u0591-\u05C7]', '', text)

def remove_numbers(text):
    """Remove numbers from text."""
    return re.sub(r'\d+', '', text)

def text_no_punctuation(text):
    """Remove punctuation from text, keeping only word characters and whitespace."""
    return re.sub(r'[^\w\s]', '', text)

def normalize_hebrew_text(text):
    """Normalize repeated characters in text."""
    pattern = re.compile(r"(.)\1{1,}", re.DOTALL)
    return pattern.sub(r"\1", text)

In [72]:
preprocessing_functions = [
    remove_html_tags,
    remove_emojis_special_chars,
    remove_diacritics,
    remove_numbers,
    text_no_punctuation,
    normalize_hebrew_text
]
for func in preprocessing_functions:
    df['transcriptConsumer'] = df['transcriptConsumer'].apply(func)

In [73]:
#Create classifiction using HebrewPsychologicalLexicons - https://github.com/natalieShapira/HebrewPsychologicalLexicons/tree/master
lexicons = Lexicons()
df_out = pd.DataFrame() #or df_in
LexiconsAPI.df_to_lexicons(df, df_out, lexicons, 'transcriptConsumer','transcriptConsumerout')
df_merged = pd.concat([df, df_out], axis=1)

In [74]:
#rename the columns by the category
df_merged = df_merged.rename(columns={
    'transcriptConsumerout_ConversationTopics_Topic076': 'transcriptConsumerout_Police',
    'transcriptConsumerout_ConversationTopics_Topic109': 'transcriptConsumerout_Law',
    'transcriptConsumerout_ConversationTopics_Topic075' : 'transcriptConsumerout_Procceses'
})


In [75]:
pattern = r'transcriptConsumerout_ConversationTopics_Topic\d+'
specific_columns_to_drop = [
    'transcriptConsumerout_Questions',
    'transcriptConsumerout_StopWords',
    'transcriptConsumerout_Top2000',
    'transcriptConsumerout_Top5000'
]
columns_to_drop = [col for col in df_merged.columns if pd.Series(col).str.contains(pattern).any()]
columns_to_drop.extend(col for col in specific_columns_to_drop if col in df_merged.columns)
# Drop these columns from the DataFrame
df_merged.drop(columns=columns_to_drop, inplace=True)

In [76]:
column_names = [
    'transcriptConsumerout_Police',
    'transcriptConsumerout_Law',
    'transcriptConsumerout_Procceses',
    'transcriptConsumerout_DataDrivenSupervised_WellBeing_Clinical',
    'transcriptConsumerout_DataDrivenSupervised_WellBeing_NonClinical',
    'transcriptConsumerout_DepressiveCharacteristics_But',
    'transcriptConsumerout_DepressiveCharacteristics_Cursing',
    'transcriptConsumerout_DepressiveCharacteristics_Direct',
    'transcriptConsumerout_DepressiveCharacteristics_I',
    'transcriptConsumerout_DepressiveCharacteristics_Indecision',
    'transcriptConsumerout_DepressiveCharacteristics_Knowing',
    'transcriptConsumerout_DepressiveCharacteristics_Meaning',
    'transcriptConsumerout_DepressiveCharacteristics_Negation',
    'transcriptConsumerout_DepressiveCharacteristics_NonFluenciesAhEm',
    'transcriptConsumerout_DepressiveCharacteristics_NonFluenciesLike',
    'transcriptConsumerout_DepressiveCharacteristics_OK',
    'transcriptConsumerout_DepressiveCharacteristics_Tentativeness',
    'transcriptConsumerout_DepressiveCharacteristics_Total',
    'transcriptConsumerout_DepressiveCharacteristics_UnknownWord',
    'transcriptConsumerout_EmotionalVariety_Amused',
    'transcriptConsumerout_EmotionalVariety_Anger',
    'transcriptConsumerout_EmotionalVariety_Anticipation',
    'transcriptConsumerout_EmotionalVariety_Anxiety',
    'transcriptConsumerout_EmotionalVariety_Ashamed',
    'transcriptConsumerout_EmotionalVariety_Calm',
    'transcriptConsumerout_EmotionalVariety_Confusion',
    'transcriptConsumerout_EmotionalVariety_Contentment',
    'transcriptConsumerout_EmotionalVariety_Disgust',
    'transcriptConsumerout_EmotionalVariety_Enthusiastic',
    'transcriptConsumerout_EmotionalVariety_Fatigue',
    'transcriptConsumerout_EmotionalVariety_Guilty',
    'transcriptConsumerout_EmotionalVariety_Hostile',
    'transcriptConsumerout_EmotionalVariety_Interested',
    'transcriptConsumerout_EmotionalVariety_Joy',
    'transcriptConsumerout_EmotionalVariety_Nervous',
    'transcriptConsumerout_EmotionalVariety_NotAmused',
    'transcriptConsumerout_EmotionalVariety_NotAnger',
    'transcriptConsumerout_EmotionalVariety_NotAnticipation',
    'transcriptConsumerout_EmotionalVariety_NotAnxiety',
    'transcriptConsumerout_EmotionalVariety_NotAshamed',
    'transcriptConsumerout_EmotionalVariety_NotCalm',
    'transcriptConsumerout_EmotionalVariety_NotConfusion',
    'transcriptConsumerout_EmotionalVariety_NotContentment',
    'transcriptConsumerout_EmotionalVariety_NotDisgust',
    'transcriptConsumerout_EmotionalVariety_NotEnthusiastic',
    'transcriptConsumerout_EmotionalVariety_NotFatigue',
    'transcriptConsumerout_EmotionalVariety_NotGuilty',
    'transcriptConsumerout_EmotionalVariety_NotHostile',
    'transcriptConsumerout_EmotionalVariety_NotInterested',
    'transcriptConsumerout_EmotionalVariety_NotJoy',
    'transcriptConsumerout_EmotionalVariety_NotNervous',
    'transcriptConsumerout_EmotionalVariety_NotProud',
    'transcriptConsumerout_EmotionalVariety_NotSad',
    'transcriptConsumerout_EmotionalVariety_NotSurprise',
    'transcriptConsumerout_EmotionalVariety_NotTrust',
    'transcriptConsumerout_EmotionalVariety_NotVigor',
    'transcriptConsumerout_EmotionalVariety_Proud',
    'transcriptConsumerout_EmotionalVariety_Sad',
    'transcriptConsumerout_EmotionalVariety_Surprise',
    'transcriptConsumerout_EmotionalVariety_Trust',
    'transcriptConsumerout_EmotionalVariety_Vigor',
    'transcriptConsumerout_Paralinguistics_Body',
    'transcriptConsumerout_Paralinguistics_Crying',
    'transcriptConsumerout_Paralinguistics_HighTone',
    'transcriptConsumerout_Paralinguistics_Humming',
    'transcriptConsumerout_Paralinguistics_ImitationTone',
    'transcriptConsumerout_Paralinguistics_Joy',
    'transcriptConsumerout_Paralinguistics_LowTone',
    'transcriptConsumerout_Paralinguistics_Sarcasm',
    'transcriptConsumerout_Paralinguistics_Sigh',
    'transcriptConsumerout_Paralinguistics_Smirk',
    'transcriptConsumerout_Paralinguistics_TutTut',
    'transcriptConsumerout_Valence_Negative',
    'transcriptConsumerout_Valence_Positive'
]

# Iterate through each column and print its unique values
for column in column_names:
    if column in df_merged.columns:  # Check if the column exists in the DataFrame
        unique_values = df_merged[column].unique()
        print(f"Unique values in {column}: {unique_values}")
    else:
        print(f"Column {column} does not exist in DataFrame.")


Unique values in transcriptConsumerout_Police: [0 2 1 3 4 5 7]
Unique values in transcriptConsumerout_Law: [ 0  1  3  6  4  2 21  7  8 12  5  9 13]
Unique values in transcriptConsumerout_Procceses: [ 1  0  3  2  4  5  6 11  7  8 12 16]
Unique values in transcriptConsumerout_DataDrivenSupervised_WellBeing_Clinical: [  6   1  79  11   0   2  12  45  13  16   8  40  52  26   3  25  30  29
   7  17  60   5  15  10  20  38  24  33  28 102   4  75  35  99  42  31
  14 127  43  18 170  61   9  19  58  62  92 114  46  34 124  37  32  27
  71  36  44  21  50  22 133  76  23  63 179  55  82  57 105 150 110 215
  51  49 141 108 173 119  87 166 137  69 120 140  48 129  67  41 172  39
  95 112  74  97 122  96  66  65  47 132 106 325  83 203 107  86  72 123
 101 138  88 154 246  77  53  93  54 163 267]
Unique values in transcriptConsumerout_DataDrivenSupervised_WellBeing_NonClinical: [  3   1  48   9   0   4   2  10   5   7  19  36  15  13  14  12  29   6
  20  45  23  51  11  30  39  47   8  21  72

In [77]:
#legal aid
df_merged['legal aid'] = (
    (df_merged['transcriptConsumerout_Police'] +
     df_merged['transcriptConsumerout_Law'] +
     df_merged['transcriptConsumerout_Procceses'] > 3
).astype(int))


In [78]:
df_merged['despair'] = (
    (df_merged['transcriptConsumerout_DepressiveCharacteristics_Negation'] +
     df_merged['transcriptConsumerout_EmotionalVariety_Sad'] +
     df_merged['transcriptConsumerout_DepressiveCharacteristics_Indecision']) > 5
).astype(int)

df_merged['loneliness'] = (
    (df_merged['transcriptConsumerout_EmotionalVariety_Sad'] +
     df_merged['transcriptConsumerout_DepressiveCharacteristics_I']) > 5
).astype(int)

df_merged['emotional overflow'] = (
    df_merged.filter(like='transcriptConsumerout_EmotionalVariety_').sum(axis=1) > 10
).astype(int)

df_merged['self blame'] = (
    (df_merged['transcriptConsumerout_DepressiveCharacteristics_I'] +
     df_merged['transcriptConsumerout_DepressiveCharacteristics_Negation']) > 5
).astype(int)

df_merged['anxiety'] = (
    (df_merged['transcriptConsumerout_EmotionalVariety_Anxiety'] +
     df_merged['transcriptConsumerout_EmotionalVariety_Nervous'] +
     df_merged['transcriptConsumerout_EmotionalVariety_NotCalm']) > 5
).astype(int)

df_merged['distrust / confusion'] = (
    (df_merged['transcriptConsumerout_EmotionalVariety_Confusion'] +
     df_merged['transcriptConsumerout_DepressiveCharacteristics_Indecision']) > 5
).astype(int)

In [79]:
df_merged['urgency'] = 0
df_merged.loc[df_merged['transcriptConsumerout_EmotionalVariety_NotCalm'] > 10, 'urgency'] = 1
df_merged.loc[df_merged['transcriptConsumerout_EmotionalVariety_Nervous'] > 10, 'urgency'] = 1


In [80]:
column_names_to_drop = [
    'transcriptConsumerout_Police',
    'transcriptConsumerout_Law',
    'transcriptConsumerout_Procceses',
    'transcriptConsumerout_DataDrivenSupervised_WellBeing_Clinical',
    'transcriptConsumerout_DataDrivenSupervised_WellBeing_NonClinical',
    'transcriptConsumerout_DepressiveCharacteristics_But',
    'transcriptConsumerout_DepressiveCharacteristics_Cursing',
    'transcriptConsumerout_DepressiveCharacteristics_Direct',
    'transcriptConsumerout_DepressiveCharacteristics_I',
    'transcriptConsumerout_DepressiveCharacteristics_Indecision',
    'transcriptConsumerout_DepressiveCharacteristics_Knowing',
    'transcriptConsumerout_DepressiveCharacteristics_Meaning',
    'transcriptConsumerout_DepressiveCharacteristics_Negation',
    'transcriptConsumerout_DepressiveCharacteristics_NonFluenciesAhEm',
    'transcriptConsumerout_DepressiveCharacteristics_NonFluenciesLike',
    'transcriptConsumerout_DepressiveCharacteristics_OK',
    'transcriptConsumerout_DepressiveCharacteristics_Tentativeness',
    'transcriptConsumerout_DepressiveCharacteristics_Total',
    'transcriptConsumerout_DepressiveCharacteristics_UnknownWord',
    'transcriptConsumerout_EmotionalVariety_Amused',
    'transcriptConsumerout_EmotionalVariety_Anger',
    'transcriptConsumerout_EmotionalVariety_Anticipation',
    'transcriptConsumerout_EmotionalVariety_Anxiety',
    'transcriptConsumerout_EmotionalVariety_Ashamed',
    'transcriptConsumerout_EmotionalVariety_Calm',
    'transcriptConsumerout_EmotionalVariety_Confusion',
    'transcriptConsumerout_EmotionalVariety_Contentment',
    'transcriptConsumerout_EmotionalVariety_Disgust',
    'transcriptConsumerout_EmotionalVariety_Enthusiastic',
    'transcriptConsumerout_EmotionalVariety_Fatigue',
    'transcriptConsumerout_EmotionalVariety_Guilty',
    'transcriptConsumerout_EmotionalVariety_Hostile',
    'transcriptConsumerout_EmotionalVariety_Interested',
    'transcriptConsumerout_EmotionalVariety_Joy',
    'transcriptConsumerout_EmotionalVariety_Nervous',
    'transcriptConsumerout_EmotionalVariety_NotAmused',
    'transcriptConsumerout_EmotionalVariety_NotAnger',
    'transcriptConsumerout_EmotionalVariety_NotAnticipation',
    'transcriptConsumerout_EmotionalVariety_NotAnxiety',
    'transcriptConsumerout_EmotionalVariety_NotAshamed',
    'transcriptConsumerout_EmotionalVariety_NotCalm',
    'transcriptConsumerout_EmotionalVariety_NotConfusion',
    'transcriptConsumerout_EmotionalVariety_NotContentment',
    'transcriptConsumerout_EmotionalVariety_NotDisgust',
    'transcriptConsumerout_EmotionalVariety_NotEnthusiastic',
    'transcriptConsumerout_EmotionalVariety_NotFatigue',
    'transcriptConsumerout_EmotionalVariety_NotGuilty',
    'transcriptConsumerout_EmotionalVariety_NotHostile',
    'transcriptConsumerout_EmotionalVariety_NotInterested',
    'transcriptConsumerout_EmotionalVariety_NotJoy',
    'transcriptConsumerout_EmotionalVariety_NotNervous',
    'transcriptConsumerout_EmotionalVariety_NotProud',
    'transcriptConsumerout_EmotionalVariety_NotSad',
    'transcriptConsumerout_EmotionalVariety_NotSurprise',
    'transcriptConsumerout_EmotionalVariety_NotTrust',
    'transcriptConsumerout_EmotionalVariety_NotVigor',
    'transcriptConsumerout_EmotionalVariety_Proud',
    'transcriptConsumerout_EmotionalVariety_Sad',
    'transcriptConsumerout_EmotionalVariety_Surprise',
    'transcriptConsumerout_EmotionalVariety_Trust',
    'transcriptConsumerout_EmotionalVariety_Vigor',
    'transcriptConsumerout_Paralinguistics_Body',
    'transcriptConsumerout_Paralinguistics_Crying',
    'transcriptConsumerout_Paralinguistics_HighTone',
    'transcriptConsumerout_Paralinguistics_Humming',
    'transcriptConsumerout_Paralinguistics_ImitationTone',
    'transcriptConsumerout_Paralinguistics_Joy',
    'transcriptConsumerout_Paralinguistics_LowTone',
    'transcriptConsumerout_Paralinguistics_Sarcasm',
    'transcriptConsumerout_Paralinguistics_Sigh',
    'transcriptConsumerout_Paralinguistics_Smirk',
    'transcriptConsumerout_Paralinguistics_TutTut',
    'transcriptConsumerout_Valence_Negative',
    'transcriptConsumerout_Valence_Positive'
]
df_merged.drop(columns=column_names_to_drop, inplace=True)

In [81]:
#create categorys
column_categories = {
    'emotional support': [
        'despair', 'loneliness', 'emotional overflow', 'self blame',
        'anxiety', 'distrust / confusion'
    ],
    'legal aid': [
        'legal aid'
    ],
    'emergency': [
        'urgency'
    ]
}
def classify_row(row):
    # Check for 'emergency' first
    if row['urgency'] == 1:
        return 'emergency'
    # Check for 'legal aid' with a specific condition
    elif row['legal aid'] == 1:
        if all(row[col] == 1 for col in column_categories['emotional support']) and row['urgency'] != 1:
            return 'emotional support'
        else:
            return 'legal aid'
    # If 'legal aid' is not 1 and 'urgency' is not 1, but any 'emotional support' column is 1
    elif any(row[col] == 1 for col in column_categories['emotional support']):
        return 'emotional support'
    # Default to 'unclassified' if none of the above conditions are met
    return 'unclassified'

# Assuming 'df' is your DataFrame
# Apply the classification function row-wise and create a new 'classification' column
df_merged['classification'] = df_merged.apply(classify_row, axis=1)

#drop all unclassified
df_merged = df_merged[df_merged['classification'] != 'unclassified'].copy()

In [82]:
# merge the classified data we got with this data
classified_df = pd.read_csv('/content/clean_final_with_urgency.csv')
# Columns unique to df2 that we want to drop
columns_to_drop = ['Unnamed: 0.1', 'new assault / new exposure', 'additional notes', 'level of suicide/ level of risk',
                   'classification 2', 'obligation to report occording law', 'Unnamed: 0', 'support for support circuls']

# Dropping the unique columns from df2
classified_df = classified_df.drop(columns=columns_to_drop)
df_merged['classification '] = df_merged['classification']
df_merged = df_merged.drop(columns=['classification','legal aid','interactionsAgentFullName'])

# Now df2_dropped contains only the common columns

columns_df1 = set(df_merged.columns)
columns_df2 = set(classified_df.columns)

# Print columns for each DataFrame
print("Columns in df1:", columns_df1)
print("Columns in df2:", columns_df2)

# Find columns that are only in df1
unique_to_df1 = columns_df1 - columns_df2
print("Columns unique to df1:", unique_to_df1)

# Find columns that are only in df2
unique_to_df2 = columns_df2 - columns_df1
print("Columns unique to df2:", unique_to_df2)

# Find common columns in both df1 and df2
common_columns = columns_df1 & columns_df2
print("Common columns:", common_columns)

Columns in df1: {'interactive', 'classification ', 'self blame', 'despair', 'distrust / confusion', 'urgency', 'transcriptConsumer', 'isPartial', 'loneliness', 'anxiety', 'firstConversation', 'emotional overflow', 'category'}
Columns in df2: {'interactive', 'classification ', 'self blame', 'despair', 'distrust / confusion', 'urgency', 'transcriptConsumer', 'isPartial', 'loneliness', 'anxiety', 'firstConversation', 'emotional overflow', 'category'}
Columns unique to df1: set()
Columns unique to df2: set()
Common columns: {'interactive', 'classification ', 'distrust / confusion', 'despair', 'self blame', 'urgency', 'transcriptConsumer', 'isPartial', 'loneliness', 'anxiety', 'firstConversation', 'emotional overflow', 'category'}


In [83]:
combined_df = pd.concat([df_merged, classified_df], ignore_index=True)

In [84]:
#combined_df = combined_df.drop(columns=['category'])
combined_df.to_csv('/content/alldatatagged.csv', encoding='utf-8')